In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
from unittest.mock import AsyncMock, Mock, patch

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from dotenv import load_dotenv
from matplotlib.colors import ListedColormap

# Make sure your dotenv file has the following defined:
load_dotenv()
GIT_FOLDER = os.environ['GIT_FOLDER']
TITILER_URL = os.environ['TITILER_URL']
TITILER_API_KEY = os.environ['TITILER_API_KEY']
API_KEY = os.environ['API_KEY']
MODEL_PATH_LOCAL = os.environ['MODEL_PATH_LOCAL']

if not (GIT_FOLDER and TITILER_URL and TITILER_API_KEY and MODEL_PATH_LOCAL):
    print("ERRROR: Failed to find all the necessary environment variables!!!")
    # Note, you must restart the kernel if you want to load new environment variables

if GIT_FOLDER not in sys.path:
    sys.path.append(GIT_FOLDER)
print(sys.path)

In [ ]:
from cerulean_cloud.models import get_model
from cerulean_cloud.tiling import TMS, offset_bounds_from_base_tiles
from cerulean_cloud.titiler_client import TitilerClient
from cerulean_cloud.cloud_run_orchestrator.clients import img_array_to_b64_image
from cerulean_cloud.cloud_run_orchestrator.schema import OrchestratorInput
from cerulean_cloud.cloud_run_orchestrator.handler import _orchestrate, get_tiler, get_titiler_client, get_roda_sentinelhub_client, get_database_engine
from cerulean_cloud.cloud_run_offset_tiles.schema import InferenceInput, PredictPayload
from cerulean_cloud.cloud_run_offset_tiles.handler import predict

In [ ]:
fastaiunet = {
    "type": "FASTAIUNET",
    "file_path": "",#"experiments/2024_03_06_18_14_31_7cls_rn101_pr256_z9_fastai_baseline_noamb/tracing_cpu_model.pt",
    "layers": ["VV"],
    "cls_map": {
        0: "BACKGROUND",
        1: "INFRA",
        2: "NATURAL",
        3: "COIN_VESSEL",
        4: "REC_VESSEL",
        5: "OLD_VESSEL",
        6: "BACKGROUND"  # HITL AMBIGUOUS, should never be output by inference_idx
    },  # inference_idx maps to class table
    "name": "ResNet101 Baseline Noamb",
    "tile_width_m": 40844,  # Used to calculate zoom
    "tile_width_px": 256,  # Used to calculate scale
    "epochs": 80,
    "thresholds": {
        "poly_nms_thresh": 0.4,
        "pixel_nms_thresh": 0.4,
        "bbox_score_thresh": 0.1,
        "poly_score_thresh": 0.01, # JONA Is this working correctly???
        "pixel_score_thresh": 0.35,
        "groundtruth_dice_thresh": 0.0
    },
    "backbone_size": 101,
    "zoom_level":9,
    "scale":2,
    # "pixel_f1": 0.0,  # TODO CALCULATE
    # "instance_f1": 0.0  # TODO CALCULATE
}

maskrcnn = {
    "type": "MASKRCNN",
    "file_path": "",#"experiments/2023_10_05_02_22_46_4cls_rnxt101_pr512_px1024_680min_maskrcnn_wd01/scripting_cpu_model.pt",
    "layers": ['VV', 'INFRA', 'VESSEL'],
    "cls_map": {'0': 'BACKGROUND', '1': 'INFRA', '2': 'NATURAL', '3': 'VESSEL'},  # inference_idx maps to class table
    "name": "ResNext 101 hires56",
    "tile_width_m": 40844,  # Used to calculate zoom
    "tile_width_px": 512,  # Used to calculate scale
    "epochs": 122,
    "thresholds": {
        "poly_nms_thresh": 0.5,
        'pixel_nms_thresh': 0.4,
        'bbox_score_thresh': 0.5,
        'poly_score_thresh': 0.1,
        'pixel_score_thresh': 0.5,
        'groundtruth_dice_thresh': 0.0},
    "backbone_size": 101,
    "zoom_level":9,
    "scale":2,
    # "pixel_f1": 0.461,  # TODO CALCULATE
    # "instance_f1": 0.47  # TODO CALCULATE
}


model_dict_predefined=maskrcnn if "maskrcnn" in MODEL_PATH_LOCAL else fastaiunet

In [ ]:
async def get_titiler_client_and_offset_tiles(sentinel_scene, offset=.33):
    payload = OrchestratorInput(**sentinel_scene)
    TitilerClient_url = os.getenv('TITILER_URL')
    titiler_client = TitilerClient(url=TitilerClient_url)
    scene_bounds = await titiler_client.get_bounds(payload.sceneid)
    tiler = TMS
    base_tiles = list(tiler.tiles(*scene_bounds, [payload.zoom], truncate=False))
    offset_tile_bounds = offset_bounds_from_base_tiles(base_tiles, offset_amount=offset)
    return titiler_client, offset_tile_bounds

In [ ]:
scene_id = "S1A_IW_GRDH_1SDV_20240204T184243_20240204T184308_052413_0656A2_1B88"
test_scene = {"sceneid": scene_id , "zoom":9, "scale":2}

titler_client , tile_bounds =  await get_titiler_client_and_offset_tiles(test_scene,offset=.66)

example_tile_a1 = tile_bounds[37] # 37 and 45
example_tile_a2 = tile_bounds[45] # 37 and 45

vv_a1 = (await titler_client.get_offset_tile(scene_id, *example_tile_a1,height=512,width=512)).transpose(2,0,1)[0]
vv_a2 = (await titler_client.get_offset_tile(scene_id, *example_tile_a2,height=512,width=512)).transpose(2,0,1)[0]

plt.imshow(vv_a1, cmap="gray")

In [ ]:
titler_client , tile_bounds =  await get_titiler_client_and_offset_tiles(test_scene,offset=.0)
example_tile_b1 = tile_bounds[44] # 44 and 52
example_tile_b2 = tile_bounds[52] # 44 and 52

vv_b1 = (await titler_client.get_offset_tile(scene_id, *example_tile_b1,height=512,width=512)).transpose(2,0,1)[0]
vv_b2 = (await titler_client.get_offset_tile(scene_id, *example_tile_b2,height=512,width=512)).transpose(2,0,1)[0]

plt.imshow(vv_b1, cmap="gray")

In [ ]:
titler_client , tile_bounds =  await get_titiler_client_and_offset_tiles(test_scene,offset=.33)
example_tile_c1 = tile_bounds[37] # 37 and 45
example_tile_c2 = tile_bounds[45] # 37 and 45

vv_c1 = (await titler_client.get_offset_tile(scene_id, *example_tile_c1,height=512,width=512)).transpose(2,0,1)[0]
vv_c2 = (await titler_client.get_offset_tile(scene_id, *example_tile_c2,height=512,width=512)).transpose(2,0,1)[0]

plt.imshow(vv_c1, cmap="gray")

In [ ]:
model = get_model(model_dict_predefined,model_path_local=MODEL_PATH_LOCAL)

inf_stack_a = [
    InferenceInput(image=img_array_to_b64_image(np.array([vv_a1]*len(model_dict_predefined["layers"])))), 
    InferenceInput(image=img_array_to_b64_image(np.array([vv_a2]*len(model_dict_predefined["layers"])))),
    ]
bounds_stack_a = [example_tile_a1, example_tile_a2][:len(inf_stack_a)]

inf_stack_b = [
    InferenceInput(image=img_array_to_b64_image(np.array([vv_b1]*len(model_dict_predefined["layers"])))), 
    InferenceInput(image=img_array_to_b64_image(np.array([vv_b2]*len(model_dict_predefined["layers"])))),
    ]
bounds_stack_b = [example_tile_b1, example_tile_b2][:len(inf_stack_b)]

inf_stack_c = [
    InferenceInput(image=img_array_to_b64_image(np.array([vv_c1]*len(model_dict_predefined["layers"])))), 
    InferenceInput(image=img_array_to_b64_image(np.array([vv_c2]*len(model_dict_predefined["layers"])))),
    ]
bounds_stack_c = [example_tile_c1, example_tile_c2][:len(inf_stack_c)]


In [ ]:
out_a = model.predict(inf_stack_a)
out_b = model.predict(inf_stack_b)
out_c = model.predict(inf_stack_c)

In [ ]:
# import matplotlib.pyplot as plt
# plt.imshow(model.deserialize(out.stack[0].json_data)['masks'][0].detach().numpy().squeeze())

In [ ]:
model = get_model(model_dict_predefined,model_path_local=MODEL_PATH_LOCAL)

fc_a = model.postprocess_tileset([out_a],bounds_stack_a)
fc_b = model.postprocess_tileset([out_b],bounds_stack_b)
fc_c = model.postprocess_tileset([out_c],bounds_stack_c)

In [ ]:
import geopandas as gpd

gdf_a = gpd.GeoDataFrame.from_features(fc_a["features"])
gdf_a.plot()
print(gdf_a)
print("________________________________________________________________")
gdf_b = gpd.GeoDataFrame.from_features(fc_b["features"])
gdf_b.plot()
print(gdf_b)
print("________________________________________________________________")
gdf_c = gpd.GeoDataFrame.from_features(fc_c["features"])
gdf_c.plot()
print(gdf_c)

In [ ]:
fc_f = model.nms_feature_reduction(
    features=[fc_a, fc_b, fc_c], min_overlaps_to_keep=0
)
gdf_f = gpd.GeoDataFrame.from_features(fc_f["features"])
gdf_f.plot()
gdf_f